# Data Cleaning - Running Metadata of the RICKD Analysis

In this notebook we will focus on cleaning the run_data_meta.csv file with what we have learned from the previous notebook.

- Coalesce `SpecInjury` and `SpecInjury2` into a single column as the main injury.
- Consolidating categorical data (i.e. Other/Other, Plantar fasciitis/Plantar fasciitis)
- Removing duplicates if they exist.
- Clean free text fields.
- Consolidate data from multiple sessions for the same subject.
- Some subjects have been registered under the same ID but they are different people. i.e. 200375
- Investigate outliers in the data.
- Investigate missing data.

# Output datasets:
- run_data_meta_cleaned.csv

In [1]:
import pandas as pd
from core.constants import RICKD_RUNNING_METADATA_FILE
from core.data_quality import standardize_free_text_col

In [2]:
run_data_meta = pd.read_csv(RICKD_RUNNING_METADATA_FILE)

In [3]:
from core.data_quality import to_lowercase
# All values in lower case helps with consolidating categories without information loss.
run_data_meta_cleaned = run_data_meta.copy()
run_data_meta_cleaned = to_lowercase(run_data_meta_cleaned)


run_data_meta_cleaned = standardize_free_text_col(run_data_meta_cleaned, ['Activities'])

# We want all missing/invalid values to be represented as None for consistency.
# There is a difference between a missing value and a valule like "other" or "no injury".
invalid_values = [
    "nan",
    "hh",
    "mm",
    "ss",
    "",
]
run_data_meta_cleaned = run_data_meta_cleaned.replace(invalid_values, None)
run_data_meta_cleaned.head()

,sub_id,datestring,filename,speed_r,age,Height,Weight,Gender,DominantLeg,InjDefn,...,SpecInjury2,Activities,Level,YrsRunning,RaceDistance,RaceTimeHrs,RaceTimeMins,RaceTimeSecs,YrPR,NumRaces
0,100433,2010-10-05 13:22:40,20101005t132240.json,1.610861,53,NaN,NaN,unknown,NaN,training volume/intensity affected,...,NaN,"walking, horseback riding, strength training",recreational,13.0,10k,NaN,NaN,NaN,NaN,NaN
1,100434,2010-11-17 13:22:40,20101117t132240.json,2.237294,51,NaN,NaN,female,NaN,training volume/intensity affected,...,NaN,"running, swimming",recreational,20.0,casual runner (no times),NaN,NaN,NaN,NaN,NaN
2,100537,2012-07-03 10:25:50,20120703t102550.json,2.127441,255,173.1,67.6,female,right,2 workouts missed in a row,...,NaN,"hiking, power walking, pilates",recreational,2.0,casual runner (no times),NaN,NaN,NaN,NaN,NaN
3,100560,2012-07-17 10:37:48,20120717t103748.json,2.657365,33,179.3,83.0,female,right,no injury,...,NaN,yoga,recreational,NaN,casual runner (no times),NaN,NaN,NaN,NaN,NaN
4,101481,2012-07-17 10:50:21,20120717t105021.json,2.625088,32,176.3,58.6,female,NaN,no injury,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# If only secondary injury is populated, it becomes the primary.
# Also only populate SpecInjury2 if it is different from SpecInjury.
print("Before:")
display(run_data_meta_cleaned[(run_data_meta_cleaned['sub_id'] == 201091)][['SpecInjury', 'SpecInjury2']])

run_data_meta_cleaned['SpecInjury'] = run_data_meta_cleaned['SpecInjury'].combine_first(run_data_meta_cleaned['SpecInjury2'])
run_data_meta_cleaned['SpecInjury2'] = run_data_meta_cleaned.apply(
    lambda row: None if row['SpecInjury2'] == row['SpecInjury'] else row['SpecInjury2'],
    axis=1
)
print("\nAfter:")
run_data_meta_cleaned[(run_data_meta_cleaned['sub_id'] == 201091)][['SpecInjury', 'SpecInjury2']]

Before:


,SpecInjury,SpecInjury2
1793,NaN,calf muscle strain



After:


,SpecInjury,SpecInjury2
1793,calf muscle strain,None


In [5]:
# We have identified that some subjects have been registered under the same ID but they are different people.
# We must separate them and give them a new ID.
# 200375 has 4 sessions with different gender and the age change between sessions does not match datestring.

subject_200375 = run_data_meta_cleaned[run_data_meta_cleaned['sub_id'] == 200375]

# Create a new subject ID for the female entries
new_subject_id = 200375 + 100000
run_data_meta_cleaned.loc[(run_data_meta_cleaned['sub_id'] == 200375) &
                         (run_data_meta_cleaned['Gender'] == 'female'), 'sub_id'] = new_subject_id

# Display the results to verify the split
print("Before:")
display(subject_200375[['sub_id', 'Gender', 'datestring', 'age']])
print("\nAfter:")
display(run_data_meta_cleaned[run_data_meta_cleaned['sub_id'].isin([200375, new_subject_id])][['sub_id', 'Gender', 'datestring', 'age']].sort_values(by='sub_id'))


Before:


,sub_id,Gender,datestring,age
1491,200375,male,2014-05-01 14:27:37,51
1492,200375,female,2014-05-02 07:30:48,27
1493,200375,female,2014-05-02 07:41:59,27
1494,200375,male,2014-05-09 07:06:51,51



After:


,sub_id,Gender,datestring,age
1491,200375,male,2014-05-01 14:27:37,51
1494,200375,male,2014-05-09 07:06:51,51
1492,300375,female,2014-05-02 07:30:48,27
1493,300375,female,2014-05-02 07:41:59,27


In [6]:
# We have identified that the age of 100234 does not match the time between sessions.
# She is 3 years older in less than 1 year between sessions.
# We will assume that the first session is the correct one and we will override the age to 41 on the second session.
subject_100234 = run_data_meta_cleaned[run_data_meta_cleaned['sub_id'] == 100234]
run_data_meta_cleaned.loc[(run_data_meta_cleaned['sub_id'] == 100234) &
                         (run_data_meta_cleaned['datestring'] == '2012-05-18 09:44:09'), 'age'] = 41
run_data_meta_cleaned[run_data_meta_cleaned['sub_id'] == 100234]

,sub_id,datestring,filename,speed_r,age,Height,Weight,Gender,DominantLeg,InjDefn,...,SpecInjury2,Activities,Level,YrsRunning,RaceDistance,RaceTimeHrs,RaceTimeMins,RaceTimeSecs,YrPR,NumRaces
1150,100234,2011-10-06 13:21:51,20090909t201123.json,2.504259,40,165.3,56.4,female,left,no injury,...,NaN,running,competitive,4.5,casual runner (no times),NaN,NaN,NaN,NaN,NaN
1355,100234,2012-05-18 09:44:09,20120518t094409.json,2.278154,41,165.0,57.3,female,right,no injury,...,NaN,"running, walking, bike, swim",recreational,NaN,casual runner (no times),NaN,NaN,NaN,NaN,NaN


In [7]:
# Create a surrogate key for each session
id_df = run_data_meta_cleaned['sub_id'].astype(str) + '_' + run_data_meta_cleaned['filename'].str.replace('.json', '')
run_data_meta_cleaned.insert(0, 'id', id_df)
run_data_meta_cleaned[['id', 'sub_id', 'filename']].head(1)


,id,sub_id,filename
0,100433_20101005t132240,100433,20101005t132240.json


In [8]:
from core.constants import RICKD_RUNNING_METADATA_CLEANED_FILE

run_data_meta_cleaned.to_csv(RICKD_RUNNING_METADATA_CLEANED_FILE, index=False)

In [9]:
run_data_meta_cleaned.head()

,id,sub_id,datestring,filename,speed_r,age,Height,Weight,Gender,DominantLeg,...,SpecInjury2,Activities,Level,YrsRunning,RaceDistance,RaceTimeHrs,RaceTimeMins,RaceTimeSecs,YrPR,NumRaces
0,100433_20101005t132240,100433,2010-10-05 13:22:40,20101005t132240.json,1.610861,53,NaN,NaN,unknown,NaN,...,NaN,"walking, horseback riding, strength training",recreational,13.0,10k,NaN,NaN,NaN,NaN,NaN
1,100434_20101117t132240,100434,2010-11-17 13:22:40,20101117t132240.json,2.237294,51,NaN,NaN,female,NaN,...,NaN,"running, swimming",recreational,20.0,casual runner (no times),NaN,NaN,NaN,NaN,NaN
2,100537_20120703t102550,100537,2012-07-03 10:25:50,20120703t102550.json,2.127441,255,173.1,67.6,female,right,...,NaN,"hiking, power walking, pilates",recreational,2.0,casual runner (no times),NaN,NaN,NaN,NaN,NaN
3,100560_20120717t103748,100560,2012-07-17 10:37:48,20120717t103748.json,2.657365,33,179.3,83.0,female,right,...,NaN,yoga,recreational,NaN,casual runner (no times),NaN,NaN,NaN,NaN,NaN
4,101481_20120717t105021,101481,2012-07-17 10:50:21,20120717t105021.json,2.625088,32,176.3,58.6,female,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
print("% of missing values per column after cleaning:")
run_data_meta_cleaned.isnull().sum() / len(run_data_meta_cleaned) * 100

% of missing values per column after cleaning:


id               0.000000
sub_id           0.000000
datestring       0.000000
filename         0.000000
speed_r          0.000000
age              0.000000
Height           0.163755
Weight           0.109170
Gender           0.000000
DominantLeg     19.213974
InjDefn          4.366812
InjJoint        12.772926
InjSide         26.528384
SpecInjury      32.096070
InjDuration     77.893013
InjJoint2       54.366812
InjSide2        55.131004
SpecInjury2     85.316594
Activities      17.248908
Level           14.683406
YrsRunning      28.220524
RaceDistance    18.449782
RaceTimeHrs     69.705240
RaceTimeMins    61.735808
RaceTimeSecs    75.709607
YrPR            76.801310
NumRaces        72.489083
dtype: float64